In [1]:
# İlgili kütüphaneleri import et
import requests
from bs4 import BeautifulSoup
import pandas as pd

# GUNCEL HAFTA değişkeni tanımla
GUNCEL_HAFTA = 1

def puan_cetveli_yazdir(hafta_sayisi):
  # GUNCEL_HAFTA değişkenini fonksiyon içinde global bir değişken olarak tanımla
  global GUNCEL_HAFTA

  # Siteye istek yolla ve gelen cevabı r değişkenine ata
  url = 'https://www.tff.org/default.aspx?pageID=198&hafta='+str(hafta_sayisi)
  try:
    r = requests.get(url,timeout=15)
    r.raise_for_status()
  except requests.exceptions.HTTPError as errh:
    print("Http Hatası:",errh)
  except requests.exceptions.ConnectionError as errc:
    print("Bağlanma Hatası:",errc)
  except requests.exceptions.Timeout as errt:
    print("Zaman aşımı hatası:",errt)
  except requests.exceptions.RequestException as err:
    print("Başka bir hata",err)

  # Gelen html sayfasının metin içeriğini al
  html_doc = r.text

  # Html sayfasını parse et
  soup = BeautifulSoup(html_doc, 'html.parser')

  # Puan cetvelinin olduğu div öğesini bul
  puan_cetveli_genel = soup.find('div', {'id':"ctl00_MPane_m_198_10561_ctnr_m_198_10561_Panel1"})

  ## print("Puan cetveli Genel:\n", puan_cetveli_genel.prettify())
  # puan_tablosu_genel'in içindeki başlık ve değerleri getir
  baslik_ve_degerler = puan_cetveli_genel.find_all('span')

  # baslik_ve_degerlerdeki başlık kısmını al
  # Bu şekilde puan tablosunun başlıklarını oluşturuyoruz.
  baslik = []
  for b in baslik_ve_degerler[0:8]:
      baslik.append(b.text)

  # baslik_ve_degerler'in içindeki değerleri al
  degerler = []
  for i in baslik_ve_degerler[8:]:
      degerler.append(int(i.text))

  # Her 8'li sayı grubu bir takımın istatistiğini göstermektedir.
  # Her 8'li sayı grubunu ayrı bir listede tut
  takım_degerler = [degerler[x:x+8] for x in range(0,len(degerler),8)]

  # Takım isimlerinin bulunduğu html öğesini getir
  takimlar = puan_cetveli_genel.find_all('a')

  # Takım isimlerini al ve takim_listesi listesine ekle
  takim_listesi = []
  for takim in takimlar:
      takim_listesi.append(takim.text)

  # takim_listesindeki her bir takımı istatistik listesindeki her 8'li grup ile eşleştir
  puan_cetveli = dict(zip(takim_listesi, takım_degerler))

  # Liglerin oynandığı hafta bilgisini bul
  # Hafta bilgisinin olduğu ilgili html öğesini bul
  ust_tablo = soup.find('div', {'id':"ctl00_MPane_m_198_10561_ctnr_m_198_10561_ht"})

  # ust_tablo'daki güncel haftayı seç
  # Güncel haftanın her zaman class'ı haftaNoOff değerini alır
  GUNCEL_HAFTA = int(ust_tablo.find('td', {'class':'haftaNoOff'}).text)
  
  # Diğer haftanın her zaman class'ı haftaNoActive değerini alır
  # Kod güncel haftaya geldiği zaman GUNCEL_HAFTA artık diğer hafta olur
  if hafta_sayisi != GUNCEL_HAFTA:
    diger_hafta = int(ust_tablo.find('td', {'class':'haftaNoActive'}).text)
  else:
    diger_hafta = GUNCEL_HAFTA
    
  # puan_cetveli_df dataframe oluştur
  puan_cetveli_df = pd.DataFrame.from_dict(data=puan_cetveli, orient='index',
                            columns=baslik)
      
  # İndex'i resetle ve ismini Takım olarak değiştir
  puan_cetveli_df = puan_cetveli_df.reset_index().rename(columns={'index':'Takım'})

  # Takım sütunundaki '"1., 2., 18." gibi numaraları boş karakter ile değiştir
  puan_cetveli_df['Takım'].replace(regex=r'^(\d+\.)', value='', inplace=True)

  # 2. sütuna "Hafta" isminde bir sütun ekle
  puan_cetveli_df.insert(1, "Hafta", hafta_sayisi)

  # İlk sütuna "Sıra" isminde bir sütun ekle
  puan_cetveli_df.insert(0, 'Sıra', range(1, len(puan_cetveli_df)+1))

  # Excel'de bir çalışma sayfası oluştur ve 
  # sonra ilk hafta puan durumu sayfasını oluştur
  # İlk hafta dışındaki diğer haftaları ayrı bir sayfa olarak ekle
  if diger_hafta == 1:
    puan_cetveli_df.to_excel('superlig_puan_durumu.xlsx', index=False, sheet_name=str(hafta_sayisi)+".hafta")
  else:
    with pd.ExcelWriter('superlig_puan_durumu.xlsx',
                        engine="openpyxl",
                        mode='a') as writer:  
      puan_cetveli_df.to_excel(writer, index=False, sheet_name=str(hafta_sayisi)+".hafta")

def puan_cetvelini_olustur():
  hafta_sayisi = 1
  while hafta_sayisi <= GUNCEL_HAFTA:
    print(f"{hafta_sayisi}.hafta yazdırıldı.")
    puan_cetveli_yazdir(hafta_sayisi)
    hafta_sayisi = hafta_sayisi + 1
  print("*******************")
  print(f"{GUNCEL_HAFTA} haftalık puan cetveli Excel'de oluşturuldu.")

In [2]:
# puan_cetvelini_olustur fonksiyonunu çağır
puan_cetvelini_olustur()

1.hafta yazdırıldı.
2.hafta yazdırıldı.
3.hafta yazdırıldı.
4.hafta yazdırıldı.
5.hafta yazdırıldı.
6.hafta yazdırıldı.
7.hafta yazdırıldı.
8.hafta yazdırıldı.
9.hafta yazdırıldı.
10.hafta yazdırıldı.
11.hafta yazdırıldı.
*******************
11 haftalık puan cetveli Excel'de oluşturuldu.
